## What is feature extraction

`Feature extraction` is different from feature selection. `Feature extraction` focuses on how to extract data from complicated data, such as text or images, to numerical features. Image processing and text are all complex structured data, traditional machine learning algorithms cannot directly process both types of data. Some preprocessing must be done from these data to extract the corresponding features and prepare for downstream tasks. (In Deep Learning, it supports end-to-end training, for example, a neural network can process raw JPEG files without any processing.)

The **sklearn** provides some functions to process the image and text. But in this lesson, we only focus on the text.

Text processing is an important field of machine learning algorithms. However, the raw data, a sequence of tokens can not be processed directly by models. We need to process the raw data and extract some kinds of numerical feature vector with a fixed size for the model. We call the general process of converting the raw text documents into numerical feature vectors as `vectorization`.

## What is the sparsity

Sparsity is a feature of natural language. For vectorization, its length is generally the size of the vocabulary in the corpus. If the size of the vocabulary is ten thousand, then the vector length is ten thousand. But for a relatively short text, because the tokens are limited, there are only a very limited number of 1's, and everything else is zero.

## How does `CountVectorizer` work

`CountVectorizer` implements both tokenization and occurrence counting in a single class. There are lots of parameters, they are all very useful. Let's have a look.

- `strip_accents`: Remove the accents.
- `lowercase`: Convert all characters to lowercase.
- `preprocessor`: A callable function to preprocess the text.
- `tokenizer`: A callable function to override default tokenizer.
- `stop_words`: Remove those very common words, such as **the**, **a**, **and**. You can pass a list of words. Or just pass **english** to use the built-in list.
- `ngram_range`: A tuple, default is **(1, 1)** which means unigrams. If you pass **(2, 2)**, it means only bigrams. If you pass **(1, 2)**, it means unigrams and bigrams.
- `analyzer`: The default value is **word**, which means the feature is based on the word. If you pass **char**, which means the feature is based on character. 
- `max_df`: Float value between 0 and 1. When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold
- `min_df`: Float value between 0 and 1. When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.
- `max_featuresint`: If not none, build a vocabulary that only considers the top max_features ordered by term frequency across the corpus.

Now, let's see how to use the **CountVectorizer**. As the code below, you only need to create a CountVectorizer object by `CountVectorizer()`, fit the corpus, and transform the corpus. The new feature would be a matrix with only 0/1.

At first, let create a corpus. In this example, it's a list of string. Each item can be considered as a short document.

In [1]:
corpus = [
    "I have an apple.",
    "The apple is red",
    "I like the apple",
    "I like the orange",
    "Apple and orange are fruit",
    "The orange is yellow"
]

Then create a **CountVectorizer** with default parameter. 

As you can see, the length of the feature is **12**. So the transformed data is also **12**. 

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

counterVec = CountVectorizer()
counterVec.fit(corpus)
print("Get all the feature names of this corpus")
print(counterVec.get_feature_names())
print("The number of feature is {}".format(len(counterVec.get_feature_names())))
corpus_data = counterVec.transform(corpus)
print("The transform data's shape is {}".format(corpus_data.toarray().shape))
print(corpus_data.toarray())

Get all the feature names of this corpus
['an', 'and', 'apple', 'are', 'fruit', 'have', 'is', 'like', 'orange', 'red', 'the', 'yellow']
The number of feature is 12
The transform data's shape is (6, 12)
[[1 0 1 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 1 1 0]
 [0 0 1 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 1 1 0 1 0]
 [0 1 1 1 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 1 1]]


The last example only use the unigrams as the feature, we can have tried both the unigrams and bigrams in this demo, just pass **ngram_range=(1, 2)**.

From the output of **get_feature_names**, you can see the word pair like **apple is**, **are fruit**. They are bigrams. Meanwhile, the length of the feature list increases to **25**

In [3]:
counterVec = CountVectorizer(ngram_range=(1, 2))
counterVec.fit(corpus)
print("Get all the feature names of this corpus")
print(counterVec.get_feature_names())
print("The number of feature is {}".format(len(counterVec.get_feature_names())))
corpus_data = counterVec.transform(corpus)
print("The transform data's shape is {}".format(corpus_data.toarray().shape))
print(corpus_data.toarray())

Get all the feature names of this corpus
['an', 'an apple', 'and', 'and orange', 'apple', 'apple and', 'apple is', 'are', 'are fruit', 'fruit', 'have', 'have an', 'is', 'is red', 'is yellow', 'like', 'like the', 'orange', 'orange are', 'orange is', 'red', 'the', 'the apple', 'the orange', 'yellow']
The number of feature is 25
The transform data's shape is (6, 25)
[[1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0]
 [0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 1]]


One question, how to get the corresponding index for one specify feature, such as **apple is**?

In [4]:
idx = counterVec.vocabulary_.get("apple is")
print("The index of 'apple is' is {}".format(idx))

The index of 'apple is' is 6


## What is TF-IDF

In the last section, we introduce a method to extract information from text, which uses the frequency of the token or token pair as the feature. However, in a large corpus, some words appear everywhere, such as **the**, **a**, **I**, and so on. It doesn't make much sense if we just use the number of times they appear as a feature. So **TF-IDF** is used to re-weight the count features into floating point values.

**TF** means term-frequency while **tf–idf** means term-frequency times inverse document-frequency.

$$\operatorname{idf}(t)=\log \frac{1+n}{1+\operatorname{df}(t)}+1$$

$n$ is the total number of documents in the corpus, and $df(t)$ is the number of documents in the document set that contain term. $t$ is the term.

There are two functions about TF-IDF, **TfidfVectorizer** and **TfidfTransformer**. You can think the **TfidfVectorizer** as the combination of **CountVectorizer** and **TfidfTransformer**.

The parameters of **TfidfVectorizer** are the same as those of **CountVectorizer**. In the example below, we use the default value.

As we can see from the output below, the number of features is also **12**. However, the value of the feature is not int but float. Because all values have been re-weighted.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf.fit(corpus)
print("Get all the feature names of this corpus")
print(tfidf.get_feature_names())
print("The number of feature is {}".format(len(tfidf.get_feature_names())))
corpus_data = tfidf.transform(corpus)
print("The transform data's shape is {}".format(corpus_data.toarray().shape))
print(corpus_data.toarray())

Get all the feature names of this corpus
['an', 'and', 'apple', 'are', 'fruit', 'have', 'is', 'like', 'orange', 'red', 'the', 'yellow']
The number of feature is 12
The transform data's shape is (6, 12)
[[0.65205671 0.         0.3868386  0.         0.         0.65205671
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.38484912 0.         0.         0.
  0.53194592 0.         0.         0.64870321 0.38484912 0.        ]
 [0.         0.         0.50568743 0.         0.         0.
  0.         0.69897098 0.         0.         0.50568743 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.66871989 0.56457928 0.         0.48380155 0.        ]
 [0.         0.51089257 0.30309169 0.51089257 0.51089257 0.
  0.         0.         0.35369727 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.51824167 0.         0.43753523 0.         0.37493445 0.63199102]]
